# Bullying recognition from Tweeter Corpus, build on recent events

## Imports 

In [166]:
# Load libraries
import pandas as pd 
from pandas import DataFrame
from pandas.tools.plotting import scatter_matrix
import matplotlib.pyplot as plt
import sklearn
from sklearn import tree 
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import requests
from geopy.geocoders import Nominatim
from geopy.distance import vincenty
import json
import difflib
import copy
from pprint import pprint
import csv
import numpy as np
import re
import csv
import json
import xlwt
import re
import warnings
warnings.filterwarnings('ignore')

## Structure 
 ### Section 1 - Functions
 ### Section 2 - Preprocessing
 ### Section 3 - Machine Learning
 ### Section 4 - Demonstration
 ### Section 5 - Location Analysis

# 1. Functions

###  The code snippet below contains the functions, used accross the project, such as the Offensiveness level calculation, two tweets similarity calculation

In [154]:
#Counting the similarity persent of two texts

import difflib
def similar(a, b):
    return difflib.SequenceMatcher(None, a, b).ratio()

#Calculating the offinsiveness level

def offensiveness_weight(TweetText):        
        templist = TweetText.split()
        offensiveWordsList=list(offensiveWords.keys())
        temp = set(offensiveWordsList).intersection(templist)
        my_list = list(temp)        
        final = 0
        result=0
        
        if len(my_list)!=0:
            for word in my_list:                
                final = float(final) + float(offensiveWords[word])        
            result = final / len(my_list)
        return result
    
#Identifying the existance of the words

def wordpresence(TweetText,wordlist):
    templist1= TweetText.split()    
    temp1 = set(wordlist).intersection(templist1) 
    my_list1 = list(temp1)    
    if len(my_list1)!=0:
        return 1
    else:
        return 0
# Counting the level of offensiveness 
def offensive_level(TweetText):
    
    nounsExistance = wordpresence(TweetText,collectivenouns)    
    pronounsExistance = wordpresence(TweetText,pronouns)
    
    if offensiveness_weight(TweetText) > 0 and (wordpresence or pronounsExistance) :    
        return (offensiveness_weight(TweetText) + 1)/6 * 100
    else:
        return (offensiveness_weight(TweetText))/6 * 100
    
#Count how many words from the wordlist parameter are in tweet
def WordCount(TweetText,wordlist):
    templist1= TweetText.split()    
    temp1 = set(wordlist).intersection(templist1) 
    my_list1 = list(temp1)     
    return len(my_list1)
#Get the coordinates by textual name of the location (by address)
def GeoLocationByTweetLocation(location):        
    try:
        geolocator = Nominatim()    
        location = geolocator.geocode(location)
        return (str(location.latitude)+";" +str(location.longitude))
    except :
        return ""
#Find the distance between location in miles      
def distance(u,v):
    return vincenty(u, v).miles
#Find the adress of the loction by coordinates
def getLocationByCoordinates(c):
    geolocator = Nominatim()
    location = geolocator.reverse(c)
    return location
def writePredictionsToFile(file1,file2):
    df = pd.read_excel(file1, sheetname='Sheet1')
    writer = pd.ExcelWriter(file2)    
    m = len(df)   
    for i in range(m):       
        a= df['Offensiveness Weight'][i]
        b=df['Offensiveness Level'][i]
        c = df['Sentimental Analysis'][i]
        df['Class'][i]=dt.predict([a,b,c]) 
       
    df.to_excel(writer,'Sheet1',index=False)
    writer.save()
    writer.close()

### 1.1  Third Party Library - Sentiment Analysis

In [119]:
import requests
# Request to third party sentimental analysis library
def SentimentalAnalysis(tweet):
    r = requests.post("http://text-processing.com/api/sentiment/", data={'text': tweet})
    label= r.json()['label']
    if label =='neutral':
        return 0
    elif label=='pos':
        return 1
    else :        
        return 2  

### 1.2 Loading Dictionaries

In [155]:
collectivenouns=[]
pronouns=[]
offensiveWords = dict() 
xl = pd.ExcelFile("Dictionary.xlsx")

df = xl.parse("OffensiveWords")
l = len(df)
for i in range(l)[1:]:
    offensiveWords[(df['Word'][i]).lower()]=df['Weight'][i]
    
df = xl.parse("Nouns")
l = len(df)
for i in range(l)[1:]:
    collectivenouns.append((df['Collective Nouns'][i]).lower()) 
    
df = xl.parse("Pronouns")
l = len(df)
for i in range(l)[1:]:
    pronouns.append(df['Personal Pronouns'][i])

### 1.3 Service Functions

#### The snippet of code below reads the file, which containes the tweets in json fromat, gatherered by streiming the twitter

In [157]:
#Reading the file
def ParseFileIntoJson(file):
    TweetsJsonArray = []
    TweetsJsonArray2 = []
    myArray = []
    i=0
    with open(file) as f:
        for line in f:
            i=i+1
            line2=line.lstrip()        
            if line2!='':                
                TweetsJsonArray.append(json.loads(line2))            
    return TweetsJsonArray

### 1.4 PreProcessing Function

Function for Preprocessing the tweets and writing the content to the exel for Manual Analysis

In [158]:

def Processing(TweetsJsonArray):
    geolocator = Nominatim()
    num=1;
    book = xlwt.Workbook()
    sheet1 = book.add_sheet("Sheet1",cell_overwrite_ok=True)
    row = sheet1.row(0)
    row.write(0, 'Text')  
    row.write(1, 'Description')    
    row.write(2, 'Offensiveness Weight')
    row.write(3, 'Offensiveness Level')
    row.write(4, 'Offensive Words Amount')
    row.write(5,'Sentimental Analysis')
    row.write(6,'Class')
    row.write(7,'Geo_Location')
    row =1
    for tweet in TweetsJsonArray:      
        
        if tweet['lang']=="en":            
            row = sheet1.row(num)             
            text=''
            if 'extended_tweet' in tweet:
                text=tweet['extended_tweet']['full_text']
            elif 'retweeted_status' in tweet:
                if 'extended_tweet' in tweet['retweeted_status']:
                    text=tweet['retweeted_status']['extended_tweet']['full_text']
                else:
                    text=tweet['retweeted_status']['text']
            else:
                text=tweet['text']
            text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','',text)
            names = tweet['user']['name']
            description = tweet['user']['description']
            PERMITTED_CHARS = "0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ_- "
            text = "".join(c for c in text if c in PERMITTED_CHARS)
            text=text.lower()
            text=text.strip()
            names="".join(c for c in names if c in PERMITTED_CHARS)
            description="".join(c for c in description if c in PERMITTED_CHARS) if description!= None else ''
            description=description.lower()
            description=description.strip() # delete beginning and ending whitespaces    
            location=""
            if 'location' in tweet['user']:
                if tweet['user']['location']!='nan' and tweet['user']['location']!='':
                    location=GeoLocationByTweetLocation(tweet['user']['location'])              
            row.write(0, text)           
            row.write(1, description)               
            row.write(7, location)
            num=num+1

    book.save("TweetCorpusExel.xls")

### 1.5  Sort the tweets in the file

In [159]:
#Sort exel file by text
def SortExel(fileIn,fileOut):
    xl = pd.ExcelFile(fileIn)
    df = xl.parse("Sheet1")
    df = df.sort_values(by="Text")
    writer = pd.ExcelWriter(fileOut)
    df.to_excel(writer,'Sheet1',index=False)
    writer.save()
    writer.close()

### 1.6 Delete repeating tweets

In [160]:
#Group the tweets with the same text
def DeleteRepititions(fileIn,fileOut):
    df = pd.read_excel(fileIn, sheetname='Sheet1')
    writer = pd.ExcelWriter(fileOut)
    currentrow=0 #first raw with unique tweet   
    tweet=str(df['Text'][currentrow])
    l = len(df)
    num=0    
    for i in range(l)[1:]:         
        num=num+1
        if df['Text'][i]==tweet or similar(df['Text'][i],tweet)>0.8:        
            if len(df['Text'][i])>len(tweet):
                df['Text'][currentrow]=df['Text'][i]             
            df['Text'][i]=''
            if str(df['Geo_Location'][i])!='nan':
                if str(df['Geo_Location'][currentrow])!='nan':
                    df['Geo_Location'][currentrow]=str(df['Geo_Location'][currentrow])+"&"+str(df['Geo_Location'][i])
                else :
                    df['Geo_Location'][currentrow]=str(df['Geo_Location'][i])
            if str(df['Description'][i])!='nan':            
                    df['Text'][currentrow]=str(df['Text'][currentrow])+" \n <::> "+str(df['Description'][i])
        else:
            if str(df['Description'][currentrow])!='nan':
                df['Text'][currentrow]=str(df['Text'][currentrow])+" \n <::> "+str(df['Description'][currentrow])
            tweet=str(df['Text'][currentrow])
            df['Offensiveness Weight'][currentrow]=offensiveness_weight(tweet)        
            df['Offensiveness Level'][currentrow]=offensive_level(tweet) 
            df['Offensive Words Amount'][currentrow]=WordCount(tweet,offensiveWords)
            df['Sentimental Analysis'][currentrow]=SentimentalAnalysis(tweet)                                    
            currentrow=i  
            tweet=str(df['Text'][currentrow])
   
    df = df[df.Text != '']   
    df.to_excel(writer,'Sheet1',index=False)
    writer.save()
    writer.close()

## 2. Data Preprocessing

Below Snippets uses Functions defined above for preprocessing 

In [161]:
# 1 Parse File with tweets json
TweetsJsonArray=ParseFileIntoJson('pythontest1.json')

In [127]:
book = xlwt.Workbook()
sheet1 = book.add_sheet("Sheet1",cell_overwrite_ok=True)
book.save("TweetCorpusExel.xls")
# 2 Process Json, Write in Exel
Processing(TweetsJsonArray)

In [74]:
# 3 Sort Tweet for repitions retrieval and retweeters gathering
SortExel('TweetCorpusExel.xls','TweetCorpusExel1.xls')
# 4 Group similar tweets
DeleteRepititions('TweetCorpusExel1.xls','TweetCorpusExel2.xls')

### 2.1 Function that prepare data for machine learning

In [128]:
#prepare data for machine learning - 
#input is Manaually labelled training set
# delete rows with Nan 
#to retain only features of training set
def shapeData(file):
    df = pd.read_excel(file, sheetname='Sheet1')
    writer = pd.ExcelWriter('TweetCorpusExelML.xlsx') 
    df=df.dropna() 
    del df['Description'] 
    del df['Geo_Location']
    del df['Text']
    del df['Offensive Words Amount']
    df.to_excel(writer,'Sheet1',index=False)
    writer.save()
    writer.close()

In [129]:
shapeData('TweetCorpusExelMC.xlsx')#passing manually classified set


# 3. Machine Learning

### 3.1 Load the dataset

In [162]:
dataset = pd.read_excel(open('TweetCorpusExelML.xlsx','rb'), sheetname='Sheet1')


### 3.2 Peek at our data

In [145]:
print(dataset.head(20))  # head

    Offensiveness Weight  Offensiveness Level  Sentimental Analysis  Class
0                    0.0             0.000000                     2      0
1                    1.5            41.666667                     0      1
2                    1.0            33.333333                     0      1
3                    0.0             0.000000                     0      0
4                    0.0             0.000000                     0      0
5                    0.0             0.000000                     1      0
6                    0.0             0.000000                     0      0
7                    2.5            58.333333                     0      1
8                    3.0            66.666667                     0      1
9                    1.0            33.333333                     0      1
10                   0.0             0.000000                     2      0
11                   1.0            33.333333                     0      1
12                   0.0 

In [146]:
print(dataset.shape) #Dimensions of Dataset

(239, 4)


In [147]:
print(dataset.describe())#Statistical Summary

       Offensiveness Weight  Offensiveness Level  Sentimental Analysis  \
count            239.000000           239.000000            239.000000   
mean               1.035565            25.069735              0.410042   
std                1.374777            30.024321              0.721273   
min                0.000000             0.000000              0.000000   
25%                0.000000             0.000000              0.000000   
50%                0.000000             0.000000              0.000000   
75%                2.000000            50.000000              1.000000   
max                5.000000           100.000000              2.000000   

            Class  
count  239.000000  
mean     0.456067  
std      0.499111  
min      0.000000  
25%      0.000000  
50%      0.000000  
75%      1.000000  
max      1.000000  


In [148]:
print(dataset.groupby('Class').size())#Class Distribution

Class
0    130
1    109
dtype: int64


### 3.3 Create a Validation Dataset



In [163]:
array = dataset.values
X = array[:,0:3]
Y = array[:,3]
validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=validation_size, random_state=seed)

In [164]:
# Test options and evaluation metric
seed = 7
scoring = 'accuracy'


### 3.4 Build Models

In [151]:
# Spot Check Algorithms
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))
# evaluate each model in turn
results = []
names = []
for name, model in models:
    kfold = model_selection.KFold(n_splits=10, random_state=seed)
    cv_results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

LR: 0.821579 (0.075704)
LDA: 0.821579 (0.075704)
KNN: 0.800526 (0.094105)
CART: 0.795263 (0.073097)
NB: 0.805789 (0.063265)
SVM: 0.816316 (0.075923)


### 3.5 Select Best Model and Make Predictions

In [152]:
dt = LogisticRegression()
dt.fit(X_train, Y_train)
predictions = dt.predict(X_validation)
print(accuracy_score(Y_validation, predictions)) # accuracy
print(confusion_matrix(Y_validation, predictions)) #Confusion Matrix
print(classification_report(Y_validation, predictions)) #Classification Report

0.854166666667
[[28  3]
 [ 4 13]]
             precision    recall  f1-score   support

        0.0       0.88      0.90      0.89        31
        1.0       0.81      0.76      0.79        17

avg / total       0.85      0.85      0.85        48



df1 = pd.read_excel(open('TweetCorpusExelML.xlsx','rb'), sheetname='Sheet1')
features = list(df1.columns[:3])
print("* features:", features, sep="\n")
y = df1["Class"]
X = df1[features]
dt = LogisticRegression()
dt.fit(X, y)

# 4. Demonstration

In [165]:
TweetText = "Book is bad"
x = offensive_level(TweetText)
y = offensiveness_weight(TweetText)
z = SentimentalAnalysis(TweetText)
print(x)
print(y)
print(z)
print(dt.predict([x,y,z]))

0.0
0
2
[ 0.]


# 5 Location Analysis

#### Once Machine is trained , we write predictions to rest of the corpus,for Further analysis 

In [89]:
#TweetCorpusExelWholeCorpus3 - preprocessed file
writePredictionsToFile('TweetCorpusExelWholeCorpus3.xls','TweetCorpusExelWholeCorpus4.xls')


<a id='5'></a>

### 5.1 Reading the geo locations into the array 

In [90]:
df = pd.read_excel('TweetCorpusExelWholeCorpus4.xls', sheetname='Sheet1')
df = df[df.Class== 1]  
df = df[df.Geo_Location!=''] 
df = df[df.Geo_Location!='nan'] 
location=df['Geo_Location']
LLTuplesLocationsArray=list()
i=0
LLTuplesLocationsArray=list()
lats1=list()
lons1=list()
for tweetLoc in location:
    if str(tweetLoc)!="nan":
        tweetLocArr=tweetLoc.split('&')        
        for t in tweetLocArr:
            ll=t.split(';')
            LLTuplesLocationsArray.append([float(ll[0]),float(ll[1])])
            lats1.append(float(ll[0]))
            lons1.append(float(ll[1]))
X = np.array(LLTuplesLocationsArray,float)

### 5.2 Performing Clustering 

In [91]:
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import pdist
import numpy as np

print (X.shape) 
plt.scatter(X[:,0], X[:,1])
Z = linkage(X,metric=distance)# - distance : miles between two spots

(1459, 2)


### 5.3 Counting Cophenetic Correlation Coefficient 

In [92]:
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist
# Cophenetic Correlation Coefficient - compares (correlates) the actual pairwise distances of all your samples to those implied by the hierarchical clustering
c, coph_dists = cophenet(Z, pdist(X, distance))
c

0.77572031731131097

### 5.4  Printing the dendogram

In [93]:
plt.title('Hierarchical Clustering Dendrogram (truncated)')
plt.xlabel('sample index or (cluster size)')
plt.ylabel('distance')
dendrogram(
    Z,
    truncate_mode='lastp',  # show only the last p merged clusters
    p=25,  # show only the last p merged clusters
    leaf_rotation=90.,
    leaf_font_size=12.,
    show_contracted=True,  # to get a distribution impression in truncated branches
)
plt.show()

In [94]:
#Repeating the steps of agglomeration for capturing the content of the clusters on each iteration
X2 = X.tolist()
X3=list(map(lambda x:[x], X2))
j=0
for c in Z:    
    X3.append(X3[int(c[0])])
    ml= copy.copy(X3[len(X3)-1])
    ml.extend(X3[int(c[1])])
    X3[len(X3)-1]=copy.copy(ml)

In [95]:
#Saving the results of the clusterization into the exel file for manual analysis
book = xlwt.Workbook()
sheet1 = book.add_sheet("Sheet1",cell_overwrite_ok=True)
row = sheet1.row(0)
row.write(0, 'first cluster')
row.write(1, 'second cluster')
row.write(2, 'distance')
row.write(3, 'Number of samples')
row.write(4, 'coordinates')
row.write(5, 'iteration')
i=1
for c in Z:
    row = sheet1.row(i)
    row.write(0, c[0])
    row.write(1, c[1])
    row.write(2, c[2])
    row.write(3, c[3])
    row.write(4, ''.join(map(str, X3[1347+i])))
    row.write(5, i)
    i=i+1
    
book.save("Clusters.xls")

In [104]:
print(Z[691])    
print(getLocationByCoordinates('44.933143,7.540121'))#515

[ 1333.  2149.     0.   500.]
57, Via Roma, None, TO, PIE, Italia


In [105]:
print (Z[1178]) 
print(getLocationByCoordinates('40.0757384,-74.4041622'))
print(getLocationByCoordinates('42.36050195,-71.2009040042125'))


[  998.          2623.            43.80701036     6.        ]
Jackson Township, Ocean County, New Jersey, United States of America
New England Preferred Properties, 362, Watertown Street, Bemis, Watertown, Middlesex, Massachusetts, 02458, United States of America


In [106]:
print (Z[1080])
print (getLocationByCoordinates('52.7954791, -0.540240286617432'))
print (getLocationByCoordinates('52.478218, -1.8930052'))

[ 2464.          2491.            14.04004292    15.        ]
Overgate Road, Swayfield CP, South Kesteven, Lincolnshire, East Midlands, England, UK
Selfridges & Co, Park Street, Digbeth, Birmingham, West Midlands Combined Authority, West Midlands, England, B12, UK


In [107]:
print (Z[1078])                              
print(getLocationByCoordinates('54.666667, -1.75'))

[  90.          522.           13.75394177    2.        ]
Sloshes Lane, Phoenix Row, County Durham, North East England, England, UK


In [108]:
print (Z[1161])
print(getLocationByCoordinates('40.7317855, -73.4481361974341'))
print(Z[887])
print (getLocationByCoordinates('39.7837304, -100.4458825'))#51 - 464
print (Z[1024])  
print(getLocationByCoordinates('22.3511148, 78.6677428'))

[ 2249.          2573.            41.17669385     8.        ]
Waverly Place, Farmingdale, Nassau County, New York, 11735, United States of America
[  873.  2345.     0.     4.]
County Road 491, Kanona, Decatur County, Kansas, United States of America
[ 1522.          1527.             2.62984501    17.        ]
SH19, Tamia, Chhindwāra, Chhindwara, Madhya Pradesh, 480559, India


In [109]:
import plotly 
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.plotly as py
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='mariamam', api_key='2Fg0f8nthJw3ZimaQ54i')
# Learn about API authentication here: https://plot.ly/python/getting-started
# Find your api_key here: https://plot.ly/settings/api
data = [go.Bar(
            x=['Italia','England, UK','New York, United States of America','Kansas, United States of America'],
            y=[501, 226,139,51]
    )]

py.iplot(data, filename='basic-bar')

In [114]:
def drawArea(coordinatesArray):
    lons=list()
    lats=list()
    for c in coordinatesArray:
        lats.append(c[0])
        lons.append(c[1])    
    m = Basemap(projection='mill',llcrnrlat=-90,urcrnrlat=90,llcrnrlon=-180,urcrnrlon=180,resolution='c') 
    x, y = m(lons,lats)
    m.drawmapboundary(fill_color='#99ffff')
    m.fillcontinents(color='#cc9966',lake_color='#99ffff')
    m.scatter(x,y,3,marker='o',color='k')
    plt.title('Locations of %s ARGO floats',fontsize=12)
    plt.figure(figsize=(100,100))
    plt.show()

### 5.5 Diagram Analysis 

In [115]:
from netCDF4 import Dataset, num2date
import time, calendar, datetime, numpy
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import urllib, os
m = Basemap(projection='mill',llcrnrlat=-90,urcrnrlat=90,llcrnrlon=-180,urcrnrlon=180,resolution='c') 
x, y = m(lons1,lats1)
m.drawmapboundary(fill_color='#99ffff')
m.fillcontinents(color='#cc9966',lake_color='#99ffff')
m.scatter(x,y,3,marker='o',color='k')
plt.title('Locations of %s ARGO floats',fontsize=12)
plt.figure(figsize=(100,100))
plt.show()

<IPython.core.display.Javascript object>